In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re

In [9]:
links_df = pd.read_csv("dfclean_1adult.csv")
#days_df = pd.read_csv("days_1adult.csv")
price_df = pd.read_csv("prices_1adult.csv")
links_df.head()

,Unnamed: 0,link,reviews,score,star,short_name
0,M Social Singapore,https://www.agoda.com/en-us/m-social-singapore...,6553,8.2,4.0,m-social-singapore
1,YOTEL Singapore Orchard Road (SG Clean Certified),https://www.agoda.com/en-us/yotel-singapore-or...,9313,8.8,0.0,yotel-singapore-orchard-road
2,Hotel G Singapore (SG Clean Certified),https://www.agoda.com/en-us/hotel-g-singapore/...,8124,8.4,0.0,hotel-g-singapore
3,Resorts World Sentosa - Hotel Michael (SG Clea...,https://www.agoda.com/en-us/resorts-world-sent...,5228,8.1,5.0,resorts-world-sentosa-hotel-michael
4,Pan Pacific Serviced Suites Beach Road (SG Cle...,https://www.agoda.com/en-us/pan-pacific-servic...,4813,8.8,0.0,pan-pacific-serviced-suites-beach-road-singapore


In [10]:
links_df.shape

(304, 6)

In [11]:
price_df.shape

(308, 31)

In [88]:
path = "/Users/joycelyn/Documents/NUSY3S1/BACT/Agoda Project/chromedriver"
driver = webdriver.Chrome(path)

In [97]:
data = []
for i in range(len(links_df)): #replace with len(links_df)
    print("iter:" + str(i+1))
    hotel = []
    url = links_df.iloc[i][1]
    
    #get hotel name
    name = links_df.iloc[i][0].strip()
    hotel.append(name) #hotel name
    
    #check if there is availability in november, if not, skip this hotel to save time
    availability = False
    nov = []
    for row in range(price_df.shape[0]):
        if name in price_df.iloc[row][0]:
            for date in range(1,31):
                if price_df.iloc[row][date] != '[]' and pd.isna(price_df.iloc[row][date]) == False:
                    nov.append(price_df.iloc[row][date])
                    availability = True
                else:
                    nov.append('NA')
            break
    if availability == False:
        continue 
        
    print(name)
    print(url)
    
    #get property type
    driver.get(url)
    time.sleep(5)
    rating = driver.find_element_by_css_selector(".IconBadge--lightgrey .IconBadge__text")
    if 'star' not in rating.text: #sometimes the same xpath gives 'agoda easycancel', need to get the option next to it
        try:
            rating = driver.find_element_by_css_selector(".IconBadge--red .IconBadge__text")
        except:
            pass
    print(rating.text)
    try:
        property_type = rating.text.split(' ',1)[1]
    except: 
        property_type = rating.text
    hotel.append(property_type) 
      
    #get availability for nov
    hotel = hotel + nov
        
    #get number of hotel stars
    if 'star' in rating.text:
        hotel.append(rating.text[0])
    else:
        hotel.append(int(links_df.iloc[i][4]))
    
    #get review score
    hotel.append(links_df.iloc[i][3])
    
    #get cleanliness score
    cleanliness = driver.find_element_by_css_selector(".gHeAiK:nth-child(3)")
    #print(cleanliness.text)
    hotel.append(cleanliness.text)
    
    #get distance from shopping
    try:
        nearest_essentials = driver.find_element_by_css_selector("#abouthotel-nearest")
        distance_mall = re.split('Shopping', nearest_essentials.text)[1]
        distance_mall = re.split('\n', distance_mall)[2]
        distance_mall_arr = re.split(' ', distance_mall)
        distance_mall = distance_mall_arr[0]
        if distance_mall_arr[1] == 'km':
            distance_mall = float(distance_mall)*1000
        #print("Distance to mall:" + str(distance_mall))
        hotel.append(int(float(distance_mall)))
    except:
        hotel.append('NA')
          
    #get distance from MRT
    try:
        distance_mrt = re.split('transportation', nearest_essentials.text)[1]
        distance_mrt = re.split('\n', distance_mrt)[2]
        distance_mrt_arr = re.split(' ', distance_mrt)
        distance_mrt = distance_mrt_arr[0]
        if 'km' in distance_mrt_arr[1]:
            distance_mrt = float(distance_mrt)*1000
        #print("Distance to mrt:" + str(distance_mrt))
        hotel.append(int(float(distance_mrt))) 
    except:
        hotel.append('NA')
    
    #get early check in 
    try:
        helpful_facts = driver.find_element_by_css_selector("#abouthotel-usefulinfo")
        check_in_from = re.split('Check-in from: |Check-in until:', helpful_facts.text)[1]
        check_in_from = re.split('\n', check_in_from)[0]
        check_in_from = check_in_from.strip()
        #print("Check-in from:" + check_in_from)
        check_in_from = re.sub(':', '', check_in_from)
        if int(check_in_from) < 1400:
            hotel.append('Y')
        else:
            hotel.append('N')
    except:
        hotel.append('N')
    
    #get late check out
    try:
        check_out_by = re.split('Check-out until: |Getting:', helpful_facts.text)[1]
        check_out_by = re.split('\n', check_out_by)[0]
        check_out_by = check_out_by.strip()
        #print("Check-out by:" + check_out_by)
        check_out_by = re.sub(':', '', check_out_by)
        if int(check_out_by) > 1200:
            hotel.append('Y')
        else:
            hotel.append('N')
    except:
        hotel.append('N')
        
    #get pay later
    fail = False
    try:
        price_includes = driver.find_element_by_css_selector(".ChildRoomsList-roomCell-featureBuckets").text
        #print(price_includes)
        if "Pay nothing" in price_includes or "Pay at the hotel" in price_includes:
            hotel.append('Y')
        else:
            hotel.append('N')
    except:
        fail = True
    if fail == True:
        continue
        
    #get free cancellation
    if "Free cancellation" in price_includes:
        hotel.append('Y')
    else:
        hotel.append('N')


    #get gym
    features = driver.find_element_by_css_selector("#abouthotel-features").text
    if "Fitness center" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
        
    #get swimming pool
    if "Swimming pool" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
        
    #get car park
    if "Car park" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
    
    #get airport transfer
    if "Airport transfer" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
    
    #get breakfast
    if "breakfast" in price_includes:
        hotel.append('Y')
    else:
        hotel.append('N')
    
    #get hygiene features (for covid-19)
    try:
        hygiene = driver.find_element_by_css_selector(".fqqSwQ , .fadeIn")
        if "Hygiene Plus" in hygiene.text:
            hotel.append('Y')
        else:
            hotel.append('N')
    except:
        hotel.append('N')
    
    #get 24h front desk
    if "Front desk [24-hour]" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
        
    #get laundry service
    if "Laundry service" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
        
    #get bathtub
    try:
        room_amenities = driver.find_element_by_css_selector(".MasterRoom-amenities").text
        if "bathtub" in room_amenities:
            hotel.append('Y')
        else:
            hotel.append('N')   
    except:
        fail = True
    if fail == True:
        continue
        
    #get balcony
    if "balcony" in room_amenities:
        hotel.append('Y')
    else:
        hotel.append('N')       
        
    #get kitchen
    if "Kitchen" in room_amenities:
        hotel.append('Y')
    else:
        hotel.append('N')    
    
    #get TV
    if "cable channels" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
        
    #get internet
    if "Free Wi-Fi in all rooms" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
    
    #get air conditioning
    if "Air conditioning" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
    
    #get ironing
    if "Ironing" in features:
        hotel.append('Y')
    else:
        hotel.append('N')
    
    #get non-smoking
    if "Non-smoking" in room_amenities:
        hotel.append('Y')
    else:
        hotel.append('N')    
    
    print(len(hotel))
    print(hotel)
    data.append(hotel)
    
df = pd.DataFrame(data, columns = ['Name','Property Type'
                                   ,'1Nov','2Nov','3Nov','4Nov','5Nov','6Nov','7Nov','8Nov','9Nov','10Nov','11Nov'
                                   ,'12Nov','13Nov','14Nov','15Nov','16Nov','17Nov','18Nov','19Nov','20Nov','21Nov'
                                   ,'22Nov','23Nov','24Nov','25Nov','26Nov','27Nov','28Nov','29Nov','30Nov'
                                   ,'Number of Stars','Review Score','Cleanliness','Distance to Mall','Distance to MRT'
                                   ,'Early Check-in (Before 3pm)','Late Check-out (After 12pm)','Pay Later'
                                   ,'Free Cancellation','Gym','Swimming Pool','Car Park','Airport Transfer'
                                   ,'Breakfast','Hygiene+ (Covid-19)','24h Front Desk','Laundry Service','Bathtub'
                                   ,'Balcony','Kitchen','TV','Internet','Air Conditioning','Ironing','Non-Smoking'])
pd.set_option("display.max_rows", None, "display.max_columns", None)
df
    

iter:1
M Social Singapore
https://www.agoda.com/en-us/m-social-singapore/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=9,16,8&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
4-star Hotel
57
['M Social Singapore', 'Hotel', '166', '166', '169', '157', '166', '217', 'NA', '169', '169', '157', '169', '166', '222', '216', '166', '166', '160', '157', '160', '222', '184', '179', '179', '169', '166', '166', '189', '184', '166', '169', '4', 8.2, '8.8', 1000, 1000, 'N', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N']
iter:2
YOTEL Singapore Orchard Road (SG Clean Certified)
https://www.agoda.com/en-us/yotel-singapore-orchard-road/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&nu

5-star Hotel
57
['InterContinental Singapore (SG Clean Certified)', 'Hotel', '297', '295', '292', '287', '292', '330', 'NA', '302', '298', '291', '291', '291', '339', '367', '358', '297', '291', '291', '297', '331', '331', '321', '298', '298', '312', '312', '314', '351', '314', '306', '5', 8.8, '9.1', 1300, 270, 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N']
iter:11
Amara Sanctuary Resort Sentosa (SG Clean Certified)
https://www.agoda.com/en-us/amara-sanctuary-resort-sentosa/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=7,8&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
5-star Resort
57
['Amara Sanctuary Resort Sentosa (SG Clean Certified)', 'Resort', '403', '300', '359', '359', '359', 'N

4-star Hotel
57
['Hotel Soloha @ Chinatown (SG Clean)', 'Hotel', '121', '121', '118', '124', '118', '127', 'NA', '122', '118', '120', '118', '122', '125', '138', '117', '127', '118', '123', '117', '126', 'NA', '121', '118', '123', '120', '121', '125', '136', '119', '120', '4', 8.8, '9.2', 1300, 270, 'N', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y']
iter:21
Hotel Mono - (SG Clean Certified)
https://www.agoda.com/en-us/hotel-mono/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=3,16&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
3-star Hotel
57
['Hotel Mono - (SG Clean Certified)', 'Hotel', '131', '129', '131', '136', 'NA', '135', 'NA', '126', '124', '125', '124', 'NA', '145', 'NA', '197', '129',

5-star Resort
57
['W Singapore - Sentosa Cove (SG Clean Certified)', 'Resort', '670', '540', '540', '540', '540', '705', 'NA', '611', '599', '540', '540', '599', '670', 'NA', '705', '611', '540', '540', '576', '764', 'NA', '740', '893', '740', '740', '740', '740', 'NA', '705', '740', '5', 8.6, '9.3', 4900, 2600, 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N']
iter:31
Grand Hyatt Singapore
https://www.agoda.com/en-us/grand-hyatt-singapore/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=16&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
5-star Hotel
57
['Grand Hyatt Singapore', 'Hotel', '341', '341', '341', '341', '341', '365', 'NA', '341', '341', '341', '341', '341', '365', '388', '341', '341'

3.5-star Hotel
57
['Aqueen Hotel Paya Lebar (SG Clean, Staycation Approved)', 'Hotel', '114', '114', '114', '114', '114', 'NA', 'NA', '153', '153', '153', '153', '153', '153', 'NA', '153', '153', '153', '153', '153', 'NA', '153', '153', '144', '144', '144', '144', 'NA', 'NA', '144', 'NA', '3', 0.0, '8.3', 6100, 450, 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N']
iter:41
The St. Regis Singapore (SG Clean Certified)
https://www.agoda.com/en-us/the-st-regis-singapore/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=16&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
5-star Hotel
57
['The St. Regis Singapore (SG Clean Certified)', 'Hotel', '446', '434', '434', '434', '434', '470', 'NA', '434', '43

...

2-star Hostel
57
['S Inn Chinatown', 'Hostel', 'NA', '23', '23', '23', '23', '20', '20', '19', '22', '22', '22', '22', '20', '20', '19', '22', '22', '22', '22', '20', '20', '19', '21', '21', '21', '21', '20', '20', '19', 'NA', '2', 8.0, '7.5', 870, 170, 'N', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'N', 'Y']
iter:267
D'Hotel Singapore (SG Clean)
https://www.agoda.com/en-us/d-hotel-singapore-sg-clean/hotel/all/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-28&childAges=&defaultChildAge=8&travellerType=-1&los=1&searchrequestid=fd2ef17b-f55c-4bce-a75b-5142041592b6
4-star Hotel
57
["D'Hotel Singapore (SG Clean)", 'Hotel', 'NA', '158', '158', '158', '158', 'NA', '158', '158', 'NA', 'NA', 'NA', 'NA', '158', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '158', 'NA', 'NA', 'NA',

2-star Hostel
57
['Beary Best Kampong Glam', 'Hostel', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '31', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '31', '31', 'NA', '31', 'NA', 'NA', 'NA', '31', '31', 'NA', '31', 'NA', 'NA', 'NA', 'NA', '2', 0.0, '8.4', 1900, 350, 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y']
iter:278
Resorts World Sentosa - Beach Villas (SG Clean Certified)
https://www.agoda.com/en-us/resorts-world-sentosa-beach-villas/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-26&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=16&los=1&searchrequestid=761cb31b-0e12-44e8-8c2d-7af14b232644
5-star Resort
iter:279
Capella Singapore Hotel
https://www.agoda.com/en-us/capella-singapore-hotel/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=

3-star Hotel
57
['Holiday Inn Express Singapore Clarke Quay (SG Clean Certified)', 'Hotel', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '173', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '173', 'NA', 'NA', 'NA', '173', '173', '3', 8.6, '9.0', 520, 500, 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y']
iter:289
Hotel Indigo Singapore Katong
https://www.agoda.com/en-us/hotel-indigo-singapore-katong/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=1,16&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
4.5-star Hotel
57
['Hotel Indigo Singapore Katong', 'Hotel', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '

4-star Hotel
57
['Hotel Miramar Singapore - SG Clean Certified', 'Hotel', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '2,404', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', '4', 0.0, '7.8', 1000, 860, 'N', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'N', 'Y']
iter:301
Conrad Centennial Singapore
https://www.agoda.com/en-us/conrad-centennial-singapore/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&isAgMse=false&ccallout=false&defdate=false&voucherMode=false&adults=1&children=0&rooms=1&maxRooms=9&checkIn=2020-11-30&childAges=&defaultChildAge=8&travellerType=-1&tspTypes=16&los=1&searchrequestid=113c6cc6-25db-42db-8343-656c8db61f29
5-star Hotel
iter:302
ZEN Rooms 1515 (SG Clean)
https://www.agoda.com/en-us/zen-rooms-1515_2/hotel/singapore-sg.html?finalPriceView=0&isShowMobileAppPrice=false&cid=-1&numberOf

,Name,Property Type,1Nov,2Nov,3Nov,4Nov,5Nov,6Nov,7Nov,8Nov,9Nov,10Nov,11Nov,12Nov,13Nov,14Nov,15Nov,16Nov,17Nov,18Nov,19Nov,20Nov,21Nov,22Nov,23Nov,24Nov,25Nov,26Nov,27Nov,28Nov,29Nov,30Nov,Number of Stars,Review Score,Cleanliness,Distance to Mall,Distance to MRT,Early Check-in (Before 3pm),Late Check-out (After 12pm),Pay Later,Free Cancellation,Gym,Swimming Pool,Car Park,Airport Transfer,Breakfast,Hygiene+ (Covid-19),24h Front Desk,Laundry Service,Bathtub,Balcony,Kitchen,TV,Internet,Air Conditioning,Ironing,Non-Smoking
0,M Social Singapore,Hotel,166,166,169,157,166,217,NA,169,169,157,169,166,222,216,166,166,160,157,160,222,184,179,179,169,166,166,189,184,166,169,4,8.2,8.8,1000,1000,N,N,N,N,Y,Y,Y,Y,N,Y,Y,Y,N,N,N,Y,Y,Y,Y,N
1,YOTEL Singapore Orchard Road (SG Clean Certified),Hotel,162,162,162,162,162,167,180,162,162,162,162,162,167,169,156,156,156,156,156,161,169,160,160,156,156,156,161,169,157,156,4,8.8,9.2,2400,190,N,N,Y,Y,Y,Y,Y,N,N,Y,Y,Y,N,N,N,Y,Y,Y,Y,Y
2,Hotel G Singapore (SG Clean Certified),Hotel,99,99,99,99,97,113,117,99,99,99,99,99,113,117,96,99,99,99,99,113,NA,99,99,99,99,99,113,117,98,99,4,8.4,8.8,1300,230,N,N,N,N,Y,N,Y,N,N,Y,Y,Y,N,N,N,Y,Y,Y,Y,Y
3,Resorts World Sentosa - Hotel Michael (SG Clea...,Resort,480,457,363,362,339,480,NA,598,539,433,433,433,539,716,598,504,363,386,363,480,NA,539,504,457,457,457,539,692,480,457,5,8.1,8.3,4800,150,N,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,N,N,N,Y,Y,Y,Y,Y
4,Pan Pacific Serviced Suites Beach Road (SG Cle...,Serviced apartment,242,241,253,231,231,347,NA,253,253,241,253,253,287,319,242,242,242,231,242,284,NA,262,262,262,250,262,287,297,250,262,5,8.8,9.2,1800,410,N,N,N,Y,Y,Y,Y,Y,N,Y,Y,Y,N,N,Y,N,Y,Y,N,N
5,Hotel Boss,Hotel,112,112,112,112,112,112,122,112,112,112,112,112,112,122,112,112,112,112,112,122,NA,112,112,112,112,112,122,131,112,112,4,7.8,8.0,2300,360,N,N,N,N,Y,Y,Y,N,N,Y,Y,Y,N,N,N,Y,Y,Y,N,Y
6,Carlton Hotel Singapore - SG Clean Certified,Hotel,230,230,230,230,230,245,268,230,230,230,230,230,262,268,230,230,230,230,230,240,NA,230,230,230,230,230,262,268,230,230,5,8.5,8.7,910,160,N,N,Y,Y,Y,Y,Y,N,N,Y,Y,Y,N,N,N,Y,Y,Y,Y,N
7,Goodwood Park Hotel (SG Clean Certified),Hotel,282,282,282,282,282,330,NA,282,282,282,282,282,330,330,282,282,282,282,282,330,NA,282,282,282,282,282,330,330,282,282,5,8.6,8.7,2400,500,N,N,Y,Y,Y,Y,Y,N,N,Y,Y,Y,N,N,Y,Y,Y,Y,Y,Y
8,PARKROYAL on Beach Road Singapore (SG Clean Ce...,Hotel,232,225,245,207,212,235,NA,205,217,216,254,217,225,253,207,217,245,217,264,235,NA,217,264,264,243,264,235,245,207,264,4,8.3,8.5,1900,330,N,N,N,Y,Y,Y,Y,Y,N,Y,Y,Y,N,N,N,Y,Y,Y,Y,N
9,InterContinental Singapore (SG Clean Certified),Hotel,297,295,292,287,292,330,NA,302,298,291,291,291,339,367,358,297,291,291,297,331,331,321,298,298,312,312,314,351,314,306,5,8.8,9.1,1300,270,N,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,N,N,Y,Y,Y,Y,N


In [100]:
df.to_csv('final_df.csv', index = True, header=True)